In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from keras import backend as K
from keras.datasets import mnist

In [4]:
batch_size = 128
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

In [7]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_data_format() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [9]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        padding='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 32)          320       
_________________________________________________________________
activation (Activation)      (None, 9, 9, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 3, 32)          9248      
_________________________________________________________________
activation_1 (Activation)    (None, 3, 3, 32)          0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 1, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 32)               

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',metrics=['accuracy'])
 

In [12]:
             

model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
469/469 [==============================] - 23s 8ms/step - loss: 1.5101 - accuracy: 0.4903 - val_loss: 0.4999 - val_accuracy: 0.8568
Epoch 2/12
469/469 [==============================] - 3s 7ms/step - loss: 0.7271 - accuracy: 0.7755 - val_loss: 0.3936 - val_accuracy: 0.8789
Epoch 3/12
469/469 [==============================] - 3s 7ms/step - loss: 0.6141 - accuracy: 0.8127 - val_loss: 0.3505 - val_accuracy: 0.8967
Epoch 4/12
469/469 [==============================] - 3s 7ms/step - loss: 0.5535 - accuracy: 0.8308 - val_loss: 0.3223 - val_accuracy: 0.9037
Epoch 5/12
469/469 [==============================] - 3s 7ms/step - loss: 0.5136 - accuracy: 0.8449 - val_loss: 0.3022 - val_accuracy: 0.9100
Epoch 6/12
469/469 [==============================] - 3s 7ms/step - loss: 0.4804 - accuracy: 0.8523 - val_loss: 0.2792 - val_accuracy: 0.9185
Epoch 7/12
469/469 [==============================] - 3s 6ms/step - loss: 0.4641 - accuracy: 0.8598 - val_loss: 0.2735 - val_accuracy: 0.9193
Epoch

In [13]:
model.save('my_model.h5')

# save the weights
model.save_weights('my_model_weights.h5')

# save the architecture
with open('my_model_architecture.json', 'w') as f:
    f.write(model.to_json())